In [ ]:
rm -rf obitools4
git clone https://github.com/metabarcoding/obitools4 

Cloning into 'obitools4'...
remote: Enumerating objects: 7574, done.
remote: Counting objects: 100% (591/591), done.
remote: Compressing objects: 100% (397/397), done.
remote: Total 7574 (delta 287), reused 384 (delta 148), pack-reused 6983 (from 2)
Receiving objects: 100% (7574/7574), 188.68 MiB | 20.47 MiB/s, done.
Resolving deltas: 100% (4721/4721), done.
Updating files: 100% (529/529), done.


In [ ]:
### Fixer la version 
cd obitools4
git reset --hard 996ec69
cd ..

HEAD is now at 996ec69 update the release notes for version 4.4.0


In [ ]:
conda install -c conda-forge compilers=1.10 make=4.4.1 go=1.24.5 zlib=1.3.1 jq=1.8.1 --yes

Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [ ]:
# CHANGER LE LIEN DU GITLAB EN GITHUB
find . -type f -exec sed -i \
  -e 's|git\.metabarcoding\.org/obitools/obitools4/obitools4|github.com/metabarcoding/obitools4|g' \
  -e 's|git\.metabarcoding\.org/org-asm/org-asm\.git|github.com/metabarcoding/org-asm.git|g' \
  -e 's|git\.metabarcoding\.org/lecasofts/go/obitools|github.com/metabarcoding/obitools4|g' \
  -e 's|https://git\.metabarcoding\.org|https://github.com/metabarcoding|g' \
  {} +


In [ ]:
export CC=$CONDA_PREFIX/bin/x86_64-conda-linux-gnu-cc
export CGO_ENABLED=1
export CGO_CFLAGS="-I$CONDA_PREFIX/include"
export CGO_LDFLAGS="-L$CONDA_PREFIX/lib"

In [ ]:
# Installation d'obitools V4
cd obitools4
make
cd ..

In [ ]:
# Recherche des chemins des inputs et formatage de l'ouput 
json_path="galaxy_inputs/galaxy_inputs.json"

# INPUT_file
input_file=$(jq -r '.inputXfile[0].path' "$json_path")

multiple_primer=$(jq -r '.multipleXprimer' "$json_path")

if [[ multiple_primer == "true "]]; then
    primer=$(jq -r '.primer[0].path' "$json_path")
fi

name=$(basename -- "$input_file")       # ex: myfile.fastq.gz
extension="${filename##*.}"                # ex: gz
filename="${filename%.*}"                  # ex: myfile.fastq ou myfile

echo "Nom de fichier (sans chemin, sans extension) : $filename"
echo "Extension détectée : $extension"


output_file="outputs/outputfile.$extension"

format=""


In [ ]:
# Variable ObiPCR Mandatory

#forward PATTERN or STRING
if [[ $multiple_primer == "false" ]]; then
    forward=$(jq -r '.forward' galaxy_inputs/galaxy_inputs.json)
    if [[ -n "$forward" && "$forward" != "null" ]]; then
        forward="--forward $forward"
        echo "variable forward présente: $forward"
    else
        forward=""
        echo "variable forward absente"
    fi
fi

#reverse PATTERN or STRING
if [[ $multiple_primer == "false" ]]; then
    reverse=$(jq -r '.reverse' galaxy_inputs/galaxy_inputs.json)
    if [[ -n "$reverse" && "$reverse" != "null" ]]; then
        reverse="--reverse $reverse"
        echo "variable reverse présente: $reverse"
    else
        reverse=""
        echo "variable reverse absente"
    fi
fi

#Max length INTEGER
if [[ $multiple_primer == "false" ]]; then
    maxl=$(jq -r '.maxlen' galaxy_inputs/galaxy_inputs.json)
    if [[ -n "$maxl" && "$maxl" != "null" ]]; then
        maxl="-L $maxl"
        echo "variable maxlen présente: $maxl"
    else
        maxl=""
        echo "variable maxlen absente"
    fi
fi

# Variable ObiPCR Optional

#allowed mismatches INTEGER 
mismatches=$(jq -r '.mismatches' galaxy_inputs/galaxy_inputs.json)
if [[ -n "$mismatches" && "$mismatches" != "null" ]]; then
    mismatches="-e $mismatches"
    echo "variable mismatches présente: $mismatches"
else
    mismatches=""
    echo "variable mismatches absente"
fi

#min length INTEGER
minl=$(jq -r '.minlen' galaxy_inputs/galaxy_inputs.json)
if [[ -n "$minl" && "$minl" != "null" ]]; then
    minl="-l $minl"
    echo "variable minlen présente: $minl"
else
    minl=""
    echo "variable minlen absente"
fi

#circular BOOLEAN
circular=$(jq -r '.circular' galaxy_inputs/galaxy_inputs.json)
if [ "$circular" = "true" ]; then
    circular="-c"
    echo "variable circular présente: $circular"
else
    circular=""
    echo "variable circular absente"
fi

#delta INTEGER -D
delta=$(jq -r '.delta' galaxy_inputs/galaxy_inputs.json)
if [[ -n "$delta" && "$delta" != "null" ]]; then
    delta="-D $delta"
    echo "variable delta présente: $delta"
else
    delta=""
    echo "variable delta absente"
fi

#only complete flanking BOOLEAN
onlyflank=$(jq -r '.onlyflank' galaxy_inputs/galaxy_inputs.json)
if [[ "$onlyflank" == "true" ]]; then
    onlyflank="-f"
    echo "variable onlyflank présente: $onlyflank"
else
    onlyflank="" 
    echo "variable onlyflank absente"
fi

In [ ]:
## Test variable mode multiple primer

if [[ multiple_primer == "true" ]]; then

    column="marker"  # Change to your desired column name

    # Get column number
    col_num=$(head -1 "$primer" | tr '\t' '\n' | grep -n "^$column$" | cut -d: -f1)

    # Fill array with values from that column (excluding header)
    readarray -t "$column" < <(awk -v col="$col_num" 'NR>1 {print $col}' "$primer")

    # Example: print all values
    for val in "${marker[@]}"; do
        echo "$val"
    done

    column="forward"  # Change to your desired column name

    # Get column number
    col_num=$(head -1 "$primer" | tr '\t' '\n' | grep -n "^$column$" | cut -d: -f1)

    # Fill array with values from that column (excluding header)
    readarray -t "$column" < <(awk -v col="$col_num" 'NR>1 {print $col}' "$primer")

    # Example: print all values
    for val in "${forward[@]}"; do
        echo "$val"
    done

    column="reverse"  # Change to your desired column name

    # Get column number
    col_num=$(head -1 "$primer" | tr '\t' '\n' | grep -n "^$column$" | cut -d: -f1)

    # Fill array with values from that column (excluding header)
    readarray -t "$column" < <(awk -v col="$col_num" 'NR>1 {print $col}' "$primer")

    # Example: print all values
    for val in "${reverse[@]}"; do
        echo "$val"
    done

    column="minl"  # Change to your desired column name

    # Get column number
    col_num=$(head -1 "$primer" | tr '\t' '\n' | grep -n "^$column$" | cut -d: -f1)

    # Fill array with values from that column (excluding header)
    readarray -t "$column" < <(awk -v col="$col_num" 'NR>1 {print $col}' "$primer")

    # Example: print all values
    for val in "${minl[@]}"; do
        echo "$val"
    done

    column="maxl"  # Change to your desired column name

    # Get column number
    col_num=$(head -1 "$primer" | tr '\t' '\n' | grep -n "^$column$" | cut -d: -f1)

    # Fill array with values from that column (excluding header)
    readarray -t "$column" < <(awk -v col="$col_num" 'NR>1 {print $col}' "$primer")

    # Example: print all values
    for val in "${maxl[@]}"; do
        echo "$val"
    done
fi

In [ ]:
# Construire les arguments formatés directement dans les tableaux

for i in "${!forward[@]}"; do
    if [[ -n "${forward[$i]}" && "${forward[$i]}" != "null" ]]; then
        forward[$i]="--forward ${forward[$i]}"
    else
        forward[$i]=""
    fi
done

for i in "${!reverse[@]}"; do
    if [[ -n "${reverse[$i]}" && "${reverse[$i]}" != "null" ]]; then
        reverse[$i]="--reverse ${reverse[$i]}"
    else
        reverse[$i]=""
    fi
done

for i in "${!maxl[@]}"; do
    if [[ -n "${maxl[$i]}" && "${maxl[$i]}" != "null" ]]; then
        maxl[$i]="--max-length ${maxl[$i]}"
    else
        maxl[$i]=""
    fi
done

for i in "${!minl[@]}"; do
    if [[ -n "${minl[$i]}" && "${minl[$i]}" != "null" ]]; then
        minl[$i]="--min-length ${minl[$i]}"
    else
        minl[$i]=""
    fi
done


In [ ]:
if [[ "$multiple_primer" == "true" ]]; then
    for i in "${!forward[@]}"; do
        output_file="outputs/obipcr_${marker[i]}.fasta"
        ./obitools4/build/obipcr ${forward[$i]} ${reverse[$i]} ${maxl[$i]} ${minl[$i]} $mismatches $circular $delta $onlyflank --no-order $input_file > "$output_file"
    done
else
    ./obitools4/build/obipcr $forward $reverse $maxl $mismatches $minl $circular $delta $onlyflank $input_file --out $output_file --no-order
fi